# Trabajo práctico 1 (Parte 1.1) - Organización de datos
### Facultad de ingeniería - Universidad de Buenos Aires


---


Alumna: Agustina Bocaccio \\
Padron: 106393 \\
Email: abocaccio@fi.uba.ar \\

---

#### Segundo cuatrimestre 2021
#### Fecha de entrega: 14/10

## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## Parte 1.1

### 34) ¿Cuántos usuarios son nativos en un idioma que no sea español?

In [ ]:
id='1RILGHMiAuB3n3Nc1n1cHshOR4QkQLDmb'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('languages.csv')

languages = pd.read_csv('languages.csv')
languages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35666 entries, 0 to 35665
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   babel_user   35666 non-null  int64 
 1   babel_lang   35662 non-null  object
 2   babel_level  35666 non-null  object
dtypes: int64(1), object(2)
memory usage: 836.0+ KB


Limpio el DataFrame descartando las filas que tienen babel_lang (el lenguaje) en null

In [ ]:
languages.dropna(inplace=True)

Filtro las filas buscando solo las que no tengan "es" (codigo del idioma español) en la columna babel_lang y que tengan una N en el babel_level, lo cual inidica que es nativo de ese idioma. Despues agrupo por usuarios ya que se repiten ids y veo cuantos son calculando el largo del dataframe. 

In [ ]:
nonspanish_speakers = languages.loc[(languages['babel_lang'] != "es") & (languages['babel_level'] == 'N'), :].groupby('babel_user')
len(nonspanish_speakers)

2568

### 5) La palabra más común entre los títulos que no sea una stopword del inglés ni español

In [ ]:
id='1Nxk1FhbEpwAqM9070I-0ESVa4wKEVMaB'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('contents.csv')

contents = pd.read_csv('contents.csv')
titles = pd.DataFrame(contents['title'])
titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4132164 entries, 0 to 4132163
Data columns (total 1 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   title   object
dtypes: object(1)
memory usage: 31.5+ MB


Veo la cantidad de datos nulos en el dataframe y posteriormente lo limpio

In [ ]:
pd.DataFrame({'Cantidad de nulos': titles.isnull().sum()})

,Cantidad de nulos
title,4


In [ ]:
titles.dropna(inplace=True)
pd.DataFrame({'Cantidad de nulos': titles.isnull().sum(), 'Porcentaje de nulos': (titles.isnull().mean() * 100)})

,Cantidad de nulos,Porcentaje de nulos
title,0,0.0


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Uso CountVectorizer para buscar la palabra mas comun de los titulos, quitando las stopwords de ingles y español

In [ ]:
stop= (stopwords.words('spanish')+ stopwords.words('english'))
vectorizer_stopwords = CountVectorizer(lowercase=True, stop_words = stop, max_features=1)
matrix = vectorizer_stopwords.fit_transform(titles['title'])

In [ ]:
most_common_word = list(vectorizer_stopwords.vocabulary_.keys())[0]
print("La palabra mas comun es: " + most_common_word )

La palabra mas comun es: categoría


### 38) Obtenga la matriz de correlación para saber idiomas distintos considerando que un usuario sabe un idioma si indicó un nivel de 1 o superior

In [ ]:
languages

,babel_user,babel_lang,babel_level
0,4502458,FR,2
1,5928200,aa,0
2,46918,ab,0
3,2050449,ab,1
4,4715583,ace,0
...,...,...,...
35661,2089885,zh-Hant-HK,1
35662,2089885,zh-Hant-TW,3
35663,5036533,zu,0
35664,5928200,zu,0


Como solo importa si el nivel del lenguaje es uno o superior, reemplazo todos los valores que son distintos de 0 por 1 para obtener la matriz de correlacion luego de conseguir la cantidad de lenguajes que sabe cada usuario (pongo los lenguajes como columnas y un uno si ese usuario lo sabe, cero si no).

In [ ]:
languages.loc[languages['babel_level'] !=  0, 'babel_level'] = 1 

In [ ]:
y = pd.get_dummies(languages['babel_lang'])
y

,FR,aa,ab,ace,acf,af,agr,akk,aln,alt,am,an,ang,ar,arc,arn,ary,ase,ast,av,avk,ay,az,azb,ba,ban,bar,bcl,be,be-tarask,bew,bg,bi,bjn,bm,bn,bo,br,bs,bug,...,tg,th,ti,tk,tl,tlh,tmr,tn,to,tpi,tr,tzh,tzm,tzo,ugy,uk,ur,uz,vec,vi,vo,vsv,wa,war,wo,wuu,xcw,yi,yua,yue,zap,zea,zh,zh-Hans,zh-Hans-CN,zh-Hant,zh-Hant-HK,zh-Hant-TW,zu,zza
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35661,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
35662,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
35663,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
35664,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
correlation_matrix = y.corr()
correlation_matrix

,FR,aa,ab,ace,acf,af,agr,akk,aln,alt,am,an,ang,ar,arc,arn,ary,ase,ast,av,avk,ay,az,azb,ba,ban,bar,bcl,be,be-tarask,bew,bg,bi,bjn,bm,bn,bo,br,bs,bug,...,tg,th,ti,tk,tl,tlh,tmr,tn,to,tpi,tr,tzh,tzm,tzo,ugy,uk,ur,uz,vec,vi,vo,vsv,wa,war,wo,wuu,xcw,yi,yua,yue,zap,zea,zh,zh-Hans,zh-Hans-CN,zh-Hant,zh-Hant-HK,zh-Hant-TW,zu,zza
FR,1.000000,-0.000028,-0.000040,-0.000040,-0.000028,-0.000093,-0.000028,-0.000028,-0.000040,-0.000028,-0.000049,-0.000217,-0.000079,-0.000390,-0.000063,-0.000122,-0.000028,-0.000049,-0.000350,-0.000028,-0.000028,-0.000063,-0.000079,-0.000040,-0.000028,-0.000028,-0.000074,-0.000040,-0.000140,-0.000028,-0.000028,-0.000159,-0.000028,-0.000040,-0.000028,-0.000028,-0.000028,-0.000056,-0.000101,-0.000040,...,-0.000040,-0.000122,-0.000028,-0.000049,-0.000089,-0.000049,-0.000028,-0.000028,-0.000028,-0.000040,-0.000190,-0.000028,-0.000028,-0.000028,-0.000028,-0.000231,-0.000074,-0.000063,-0.000093,-0.000093,-0.000028,-0.000040,-0.000040,-0.000040,-0.000028,-0.000028,-0.000028,-0.000084,-0.000040,-0.000097,-0.000028,-0.000040,-0.000415,-0.000028,-0.000049,-0.000028,-0.000028,-0.000028,-0.000040,-0.000028
aa,-0.000028,1.000000,-0.000040,-0.000040,-0.000028,-0.000093,-0.000028,-0.000028,-0.000040,-0.000028,-0.000049,-0.000217,-0.000079,-0.000390,-0.000063,-0.000122,-0.000028,-0.000049,-0.000350,-0.000028,-0.000028,-0.000063,-0.000079,-0.000040,-0.000028,-0.000028,-0.000074,-0.000040,-0.000140,-0.000028,-0.000028,-0.000159,-0.000028,-0.000040,-0.000028,-0.000028,-0.000028,-0.000056,-0.000101,-0.000040,...,-0.000040,-0.000122,-0.000028,-0.000049,-0.000089,-0.000049,-0.000028,-0.000028,-0.000028,-0.000040,-0.000190,-0.000028,-0.000028,-0.000028,-0.000028,-0.000231,-0.000074,-0.000063,-0.000093,-0.000093,-0.000028,-0.000040,-0.000040,-0.000040,-0.000028,-0.000028,-0.000028,-0.000084,-0.000040,-0.000097,-0.000028,-0.000040,-0.000415,-0.000028,-0.000049,-0.000028,-0.000028,-0.000028,-0.000040,-0.000028
ab,-0.000040,-0.000040,1.000000,-0.000056,-0.000040,-0.000132,-0.000040,-0.000040,-0.000056,-0.000040,-0.000069,-0.000307,-0.000112,-0.000551,-0.000089,-0.000173,-0.000040,-0.000069,-0.000495,-0.000040,-0.000040,-0.000089,-0.000112,-0.000056,-0.000040,-0.000040,-0.000105,-0.000056,-0.000198,-0.000040,-0.000040,-0.000224,-0.000040,-0.000056,-0.000040,-0.000040,-0.000040,-0.000079,-0.000143,-0.000056,...,-0.000056,-0.000173,-0.000040,-0.000069,-0.000125,-0.000069,-0.000040,-0.000040,-0.000040,-0.000056,-0.000269,-0.000040,-0.000040,-0.000040,-0.000040,-0.000327,-0.000105,-0.000089,-0.000132,-0.000132,-0.000040,-0.000056,-0.000056,-0.000056,-0.000040,-0.000040,-0.000040,-0.000119,-0.000056,-0.000137,-0.000040,-0.000056,-0.000587,-0.000040,-0.000069,-0.000040,-0.000040,-0.000040,-0.000056,-0.000040
ace,-0.000040,-0.000040,-0.000056,1.000000,-0.000040,-0.000132,-0.000040,-0.000040,-0.000056,-0.000040,-0.000069,-0.000307,-0.000112,-0.000551,-0.000089,-0.000173,-0.000040,-0.000069,-0.000495,-0.000040,-0.000040,-0.000089,-0.000112,-0.000056,-0.000040,-0.000040,-0.000105,-0.000056,-0.000198,-0.000040,-0.000040,-0.000224,-0.000040,-0.000056,-0.000040,-0.000040,-0.000040,-0.000079,-0.000143,-0.000056,...,-0.000056,-0.000173,-0.000040,-0.000069,-0.000125,-0.000069,-0.000040,-0.000040,-0.000040,-0.000056,-0.000269,-0.000040,-0.000040,-0.000040,-0.000040,-0.000327,-0.000105,-0.000089,-0.000132,-0.000132,-0.000040,-0.000056,-0.000056,-0.000056,-0.000040,-0.000040,-0.000040,-0.000119,-0.000056,-0.000137,-0.000040,-0.000056,-0.000587,-0.000040,-0.000069,-0.000040,-0.000040,-0.000040,-0.000056,-0.000040
acf,-0.000028,-0.000028,-0.000040,-0.000040,1.000000,-0.000093,-0.000028,-0.000028,-0.000040,-0.000028,-0.000049,-0.000217,-0.000079,-0.000390,-0.000063,-0.000122,-0.000028,-0.000049,-0.000350,-0.000028,-0.000028,-0.000063,-0.000079,-0.000040,-0.000028,-0.000028,-0.000074,-0.000040,-0.000140,-0.000028,-0.000028,-0.000159,-0.000028,-0.000040,-0.000028,-0.000028,-0.000028,-0.000056,-0.000101,-0.000040,...,-0.000040,-0.000122,-0.000028,-0.000049,-0.00

### 16) Calcule la probabilidad de las palabras para los textos, luego encuentre el documento que más se desvie de esas probabilidades utilizando la divergencia de Kullback-Leibler

In [ ]:
id='14B07LGSE60-P423xMrTVQAzGvLXsqm_2'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('contents_text_sample.csv')

contents_text_samples = pd.read_csv('contents_text_sample.csv')

In [ ]:
contents_text_samples

,id,title,text
0,50,Anacardiaceae,{{Ficha de taxón\n| name = Anacardiáceas\n| im...
1,62,Antipartícula,[[Archivo:Materia antimateria.png|thumb|200px|...
2,78,Año bisiesto,'''Año bisiesto''' es una expresión que deriva...
3,127,Avena,{{otros usos|Avena sativa|la avena común}}\n{{...
4,160,Alfabeto,{{distinguir|analfabeto|alfabetización}} \n[[A...
...,...,...,...
206602,10010394,Nicoleta-Ancuţa Bodnar,#REDIRECCIÓN [[Nicoleta-Ancuța Bodnar]]
206603,10010425,Sebastian Szczepański,{{Ficha de deportista\n|nombre = ...
206604,10010518,Manu (Tripura),{{Ficha de entidad subnacional\n|nombre comple...
206605,10010570,ISRU,#REDIRECCIÓN [[Utilización de recursos in situ]]


Obtengo una matriz que tiene a todas las palabras que aparecen como columnas y como filas a los textos. Dentro tiene que cantidad de veces aparece cada palabra en cada texto. 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(lowercase=True)
contents_text_samples.dropna(inplace=True)
matrix = vectorizer.fit_transform(contents_text_samples.text[:1000]) #tomo los primeros 10000 textos por falta de ram

In [ ]:
#NxK N es la cantidad de textos (206600), y K es la cantidad de palabras
matrix

<1000x153465 sparse matrix of type '<class 'numpy.int64'>'
	with 753956 stored elements in Compressed Sparse Row format>

In [ ]:
vectorizer.vocabulary_
sum_words = matrix.sum(axis=0) 

In [ ]:
sum_words = np.asarray(matrix.sum(axis=0))
sum_words.shape

(1, 153465)

Calculo la probabilidad total de las palabras 

In [ ]:
prob_words = (sum_words/sum_words.sum())
prob_words

array([[1.01952552e-04, 4.45139292e-04, 2.40832800e-06, ...,
        4.01388000e-07, 4.01388000e-07, 4.01388000e-07]])

Busco la probabilidad de las palabras para cada texto: 

In [ ]:
def array_mean(x):
  y = x/x.sum()
  return y

prob_each_word_in_texts = np.apply_along_axis(array_mean, 1, matrix.toarray())
prob_each_word_in_texts

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00054855, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.00090334, 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
prob_each_word_in_texts.shape, prob_words.shape, prob_each_word_in_texts.shape[0]

((1000, 153465), (1, 153465), 1000)

Busco el texto que mas se aleje de la probabilidad total de las palabras usando la divergencia de Kullback-Leibler

In [ ]:
#para usar la divergencia de Kullback-Leibler
import scipy.special as scipy
div= scipy.rel_entr(prob_each_word_in_texts, prob_words).sum(axis=1)
contents_text_samples.loc[np.argmax(div), ['title']] 

title    MediaWiki:Unblocklogentry
Name: 639, dtype: object

### 23) ¿Qué porcentaje de los contenidos contienen a su mismo título en el texto? 

In [ ]:
pd.DataFrame({'Cantidad de nulos': contents_text_samples.isnull().sum(), 'Porcentaje de nulos': (contents_text_samples.isnull().mean() * 100)})

,Cantidad de nulos,Porcentaje de nulos
id,0,0.0
title,0,0.0
text,0,0.0


In [ ]:
#filtrado, no cuento los contents con texto nulo
contents_text_samples.dropna(inplace= True)
contents_text_samples.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 206600 entries, 0 to 206606
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      206600 non-null  int64 
 1   title   206600 non-null  object
 2   text    206600 non-null  object
dtypes: int64(1), object(2)
memory usage: 6.3+ MB


Veo para cada fila si el titulo esta en el texto que le corresponde



In [ ]:
is_title_in_text = contents_text_samples.apply(lambda row: row['title'].lower() in row['text'].lower(), axis=1)
print("Porcentaje de documentos que tienen a su titulo en su texto:" )
is_title_in_text.mean()*100

Porcentaje de documentos que tienen a su titulo en su texto:


38.3809293320426